# SHARK QUEST
## New Kids On The Block



# TODO

- Pickle ?
- Column names?

## Selecting
- Select relevant columns
- Analyse relevant columns

## Cleaning :
- Cast to appropriate data types
    - General cleaning
        - identify duplicates
            - fuzzy
        - removing duplicates
            - remove
            - merge
        - handling null values
            - remove
            - replace
        - manipulating strings
        - formatting the data.

- Wrong inputs
- Outliers

## Hypothesis

### TIME
- Shark attacks are seasonal (Summer)
- Shark attacks are increasing 
- Shark attacks are more concentrated in the PM 

### LOCATION
- Some countries are more attack prone (Australia)
- Some countries are more likely to be fatal (Australia)


### DEMOGRAPHICS
- Males are more likely to get attacked
- Males are more likely to get provoke a shark
- Provoked attacked are more fatal
- Young persons are more likely to get attacked
- Old persons are more likely to get killed

- Names more likely to get attacked (John)

### OTHER
- Some species are more aggressive (Tiger Shark)
- Some activities are more likely (Surfing)
- Some activities are more fatal

- Full moon? 😂


## Business Ideas

- App that gives a likelyhood of attack based on location and time
- Vacations far away from sharks for phobics
- Witness attacks for masochists

- Surf school at the safest places / seasons
- Fishing supplies => shark repellant by activities
- Safety training to avoid provocations / live in harmony

- Shark repellant => best spots
- Insurance for surfers, premiums for high risk areas

In [ ]:
import pandas as pd
import re

# Load the data
shark_attacks = pd.read_excel('GSAF5.xls')
display(shark_attacks)

#new_data.to_csv("shark_data.csv")

In [ ]:
# Settings
pd.set_option('display.max_rows', 8000)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 100)


In [31]:
data_schema = {
    'Type': {'dtype': 'category', 'categories': ['PROVOKED', 'UNPROVOKED', 'INVALID']},
    'Date': {'dtype': 'datetime64[ns]'},
    'Country': {'dtype': 'object'},
    'State': {'dtype': 'object'},
    'Location': {'dtype': 'object'},
    'Activity': {'dtype': 'category'},
    'Injury': {'dtype': 'category', 'categories': ['FATALITY', 'INJURY', 'OTHER']},
    'Time': {'dtype': 'category', 'categories': ['MORNING', 'NOON', 'AFTERNOON', 'DANW', 'NIGHT', 'DUSK']}
}

In [ ]:
# RENAME COLUMNS
""" def rename_columns(column_name:str, no_space=True, lowercase=True):
    new_name = column_name
    print(new_name)
    if no_space:
        new_name = column_name.replace(r'\s', '-')

    if lowercase:
        new_name = new_name.lower()
    
    return new_name


shark_attacks.rename(rename_columns, axis=1)
shark_attacks """

# FUNCTIONS

In [ ]:
#Hennings functions
    #Merge categories : (*categories to be merged, target) - Henning


In [34]:
#Linh functions
    #Strip function : strips spaces - Linh
def strip_func(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    return df

df_cleaned = strip_func(shark_attacks)
df_cleaned.head(1000)


In [ ]:
df_cleaned['Location'] = df_cleaned['Location'].astype(str)

def clean_data(location):
    location = re.sub(r'\d{1,2}º[NS],\s*\d{1,3}º[EW]', '', location)
    location = re.sub(r'\(.*?\)', '', location)
    location = re.sub(r'\s+', ' ', location)
    return location.title()

df_cleaned['Location'] = df_cleaned['Location'].apply(clean_data)
df_cleaned['Location'].head(1000)


In [ ]:
categories = [(r"swimming|wading","swimming")]

def categorize_activity(activity):

    if pd.isna(activity):
        return "Invalid"
    

    activity = activity.lower()
    
    if 'swim' in activity or 'wading' in activity:
        return "Swimming"
    elif 'surf' in activity or 'boogie boarding' in activity or 'paddle' in activity or 'foil' in activity:
        return "Surfing"
    elif 'fish' in activity or 'fishing' in activity or 'spearfish' in activity or 'kayak' in activity:
        return "Fishing"
    elif 'diving' in activity or 'scuba' in activity or 'freediving' in activity or 'snorkel' in activity:
        return "Diving"
    elif 'sit' in activity or 'stand' in activity:
        return "Passive"
    else:
        return "Other Activity" 


df_cleaned['Activity'] = df_cleaned['Activity'].apply(categorize_activity)
df_cleaned.head(100)


In [35]:
#Ricardo functions
    #Replace as Nan, Nat, ... function - Ricardo
    #Remove obvious duplicate (entire line) - Ricardo
    #Remove fuzzy duplicates (case number? dates?) - Ricardo


In [36]:
# Jp functions

# replacements
replacements_test_species = [
    (r'^.*([Tt]iger).*$', 'TIGER SHARK'),
]

replacements_test_dates = [
    (r'^(.*)(\d{2})(.*)(\w{3})(.*)(\d{4})(.*)', r'\2-\4-\6'),
    (r'^(.*)(\w{3})(.*)(\d{4})(.*)', r'01-\2-\4'),
    (r'-(uly|une)', r'-J\1'),
]

replacements_test_time = [
    (r'^.*((0[0-2]|23)h|([nN]ight|[dD]ark)).*$', 'NIGHT'),
    (r'^.*((0[3-6])h|([dD]awn|[sS]unrise|[dD]aybreak)).*$', 'DAWN'),
    (r'^.*((0[7-9]|10)h.*$|([mM]orning|^[aA]\.?[mM])).*$', 'MORNING'),
    (r'^.*((1[1-4])h.*$|([nN]oon|[mM]idday|[lL]unch)).*$', 'NOON'),
    (r'^.*((1[5-8])h.*$|([aA]fternoon|^[pP]\.?[mM])).*$', 'AFTERNOON'),
    (r'^.*((19|2[0-2])h.*$|([dD]usk|[sS]unset|[eE]vening)).*$', 'DUSK'),
]

In [ ]:

def replace_string_patterns(value, replacements):

    if isinstance(value, str):

        for pattern, target in replacements:
            value = re.sub(pattern, target, value)
        return value
    
    else:
        
        return value

species_replace = shark_attacks['Species '].apply(replace_string_patterns, replacements = replacements_test_species)
display(species_replace)

In [ ]:
#DATE TEST
date_replace_test = shark_attacks['Date'].apply(replace_string_patterns, replacements = replacements_test_dates,)
date_test = pd.to_datetime(date_replace_test, format='mixed', errors='coerce')

formatted_dates = date_test.dt.strftime('%d-%m-%Y')
formatted_dates = formatted_dates.astype('datetime64[ns]')
formatted_dates

In [ ]:
#TIME TEST
display(shark_attacks['Time'].value_counts())
time_replace_test = shark_attacks['Time'].apply(replace_string_patterns, replacements = replacements_test_time,)
display(time_replace_test.value_counts())
time_replace_test

In [ ]:
valid_categories = [value for key, value in replacements_test_time]
#PRINT ALL COLUMNS WITH FIRST VALUE
shark_attacks.iloc[0]

In [ ]:
# Source, PDF, Case Number as possible duplicate finder or year / date fill
relevant_columns = ['Date','Year','Type','Country','State','Location','Activity','Name','Sex','Age','Injury','Time','Species','Source', 'pdf','href formula','href' ]
business_relevant_columns = ['Date','Type','Country','State','Location','Activity','Injury','Time']

#Henning : Date, Type
#Ricardo : Country, State
#Linh : Location, Activity
#Jp : Injury, Time
 
#Type : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Provoked, Unprovoked, Invalid
#Date : datetime : Clean "Reported" - Harmonize Format - Cast weird into NaT. Final => Dates (as datetime), NaT
#Country : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
#State : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
#Location : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
#Activity : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Few categories to be determined
#Injury : category : Merge columns based on keywords. Nan into other. Final => Fatality, Injury, Other
#Time : category : Cast into categories Final => morning (6-10) noon (10-14) afternoon( 14-18) dusk (18-22) night (22 - 2) dawn (2-6) maybe as integers (0-5)

display(time_replace_test.value_counts())
where_test = time_replace_test.where(time_replace_test.isin(valid_categories))
where_test = where_test.astype('category')
where_test


In [ ]:
#RENAME COLUMNS

In [ ]:
# RETYPE COLUMNS
columns_types = {key: value['dtype'] for key, value in data_schema.items()}
print(columns_types)
shark_attacks.astype(columns_types)

In [ ]:
shark_attacks.shape

In [ ]:
shark_attacks.head()      

In [ ]:
#PRINT ALL COLUMNS WITH FIRST VALUE
shark_attacks.iloc[0]

In [69]:
# Source, PDF, Case Number as possible duplicate finder or year / date fill
relevant_columns = ['Date','Year','Type','Country','State','Location','Activity','Name','Sex','Age','Injury','Time','Species','Source', 'pdf','href formula','href' ]
business_relevant_columns = ['Date','Type','Country','State','Location','Activity','Injury','Time']

#Henning : Date, Type
#Ricardo : Country, State
#Linh : Location, Activity
#Jp : Injury, Time
 
#Type : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Provoked, Unprovoked, Invalid
#Date : datetime : Clean "Reported" - Harmonize Format - Cast weird into NaT. Final => Dates (as datetime), NaT
#Country : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
#State : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
#Location : string : Strip spaces - Formatting - Replace weird characters - cast weird values as NaN => Strings, Nan
#Activity : category : Merge some columns based on categories. Trim labels. Nan into invalid. Final => Few categories to be determined
#Injury : category : Merge columns based on keywords. Nan into other. Final => Fatality, Injury, Other
#Time : category : Cast into categories Final => morning (6-10) noon (10-14) afternoon( 14-18) dusk (18-22) night (22 - 2) dawn (2-6) maybe as integers (0-5)

#Todo
    #Remove obvious duplicate (entire line) - Ricardo
    #Remove fuzzy duplicates (case number? dates?) - Ricardo

#Functions : 
    #Merge categories : (*categories to be merged, target) - Henning
    #Strip function : strips spaces - Linh
    #Replace as Nan, Nat, ... function - Ricardo
    # DONE - Replace by keyword function - Jp
    # DONE - Cast to dateTime function 
        #Matching function (find similarities, keyword based?)
        # DONE - Reformat dates, strings
        #Filter function

In [ ]:
matching_rows = shark_attacks[shark_attacks['Injury'].str.contains('FATAL', case=False, na=False)]

print(matching_rows)

In [ ]:
#PRINT MULTIPLE COLUMNS

display(shark_attacks[['Case Number','Case Number.1']].dropna())

In [ ]:
pattern = r"Unnamed"
unnamed_columns = shark_attacks[shark_attacks.filter(regex=pattern).columns].dropna(thresh=1)
display(unnamed_columns.head(100))


In [ ]:
# PRINT ALL MISFORMATED DATES (YEAR END \d{4})
shark_attacks[~shark_attacks['Date'].str.contains(r'.*\d{4}$', regex=True, na=False)]

In [ ]:
shark_attacks['Name'].value_counts()

def get_fname(row):
    return str(row).split()[0]

shark_attacks['first_name'] = shark_attacks['Name'].apply(get_fname)

shark_attacks['first_name'].value_counts().head(50)


### INJURY

## Problems
- Some are non-human damages (boat, kayak, boards)
- Some are non injuries (touched shark)
- Some don't have details
- Fatalities flags are non consistent
- Some incident are post-mortem
- Some are not incidents ('Aboriginal rock carving depicts man being attacked by a shark')
- Nan

## Additional
- Some provoked details that could be useful if missing elsewhere

## Solutions
- 3 categories : 
    - Fatality
    - Injury
    - Other

- Keyword(s) based search / replace ?

### TIME

## Problems
- Lots of Nan
- Some are times (12h00,...), other are time of day (morning,...)
- Some are text
- Some are ranges
- Some are calculations
- Some are 12h based, other 24h based
- Some maybe duplicates 
- Some are misplaced data

## Solutions
- 2 types :
    - Time (int? str?)
    - Unknown / NaN / None
